PART E

In [2]:
import gurobipy as gb
from gurobipy import GRB
import pandas as pd

# Read the data
df = pd.read_csv('/Users/huiyisang/Desktop/Modeling A2/hotels.csv')

# Create model
model = gb.Model("Hotel_Staff_Optimization")

# Constants
num_attendants = 8
hourly_wage = 25
regular_hours = 8
max_overtime = 2
sq_ft_limit = 3500
floor_bonus = 75


# Get unique floors and create floor mapping
floors = sorted(df['Floor'].unique())
floor_to_idx = {floor: idx for idx, floor in enumerate(floors)}
num_floors = len(floors)

# Create variables
# xir = 1 if attendant i is assigned to room r
x = model.addVars(num_attendants, len(df), vtype=GRB.BINARY, name="room_assignment")

# fik = 1 if attendant i works on floor k
f = model.addVars(num_attendants, num_floors, vtype=GRB.BINARY, name="floor_assignment")

# oi = number of overtime hours for attendant i (0, 1, or 2)
o = model.addVars(num_attendants, max_overtime, vtype=GRB.BINARY, name="overtime")

# si = 1 if attendant i exceeds square footage limit
s = model.addVars(num_attendants, vtype=GRB.BINARY, name="sqft_violation")


# Objective function
obj = (sum(regular_hours * hourly_wage for i in range(num_attendants)) +  # Base pay
       sum(hourly_wage * 1.5 * (o[i,0] + o[i,1]) for i in range(num_attendants)) +  # Overtime
       sum(floor_bonus * (sum(f[i,k] for k in range(num_floors)) - 2) 
           for i in range(num_attendants)) +  # Floor bonus
       sum(regular_hours * 2 * hourly_wage * s[i] for i in range(num_attendants)))  # Square footage violation

model.setObjective(obj, GRB.MINIMIZE)


# Constraints
# Each room must be assigned to exactly one attendant
for j in range(len(df)):
    model.addConstr(sum(x[i,j] for i in range(num_attendants)) == 1)

# Link floor assignments to room assignments
for i in range(num_attendants):
    for floor in floors:
        floor_idx = floor_to_idx[floor]
        room_indices = df[df['Floor'] == floor].index
        model.addConstr(sum(x[i,j] for j in room_indices) <= len(room_indices) * f[i,floor_idx])
        model.addConstr(sum(x[i,j] for j in room_indices) >= f[i,floor_idx])

# Square footage constraint with calculated big M
M = sum(df.Square_Feet)
for i in range(num_attendants):
    model.addConstr(sum(df.loc[j,'Square_Feet'] * x[i,j] for j in range(len(df))) <= 
                    sq_ft_limit + M * s[i])

# Overtime constraints
for i in range(num_attendants):
    model.addConstr(sum(df.loc[j,'Cleaning_Time_Hours'] * x[i,j] for j in range(len(df))) <= 
                    regular_hours + o[i,0] + o[i,1])
    model.addConstr(o[i,1] <= o[i,0])  # Can't work 2nd overtime hour without working 1st

# Floor limit constraint (between 2 and 4 floors)
for i in range(num_attendants):
    model.addConstr(sum(f[i,k] for k in range(num_floors)) >= 2)  # minimum 2 floors
    model.addConstr(sum(f[i,k] for k in range(num_floors)) <= 4)  # maximum 4 floors

# Optimize
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    print(f"\nOptimal Cost: ${model.objVal:.2f}")
    print(f"Big M value used: {M:.2f}")
    
    # Count overtime hours
    total_overtime = sum(o[i,j].x for i in range(num_attendants) for j in range(max_overtime))
    print(f"Total Overtime Hours: {total_overtime}")
    
    # Count floor violations
    floor_violations = sum(max(0, sum(f[i,k].x for k in range(num_floors)) - 2) 
                         for i in range(num_attendants))
    print(f"Total Floor Violations: {floor_violations}\n")
    
    print("Detailed Attendant Statistics:")
    print("-" * 80)
    print(f"{'Attendant':^10} {'Hours':^10} {'Rooms':^10} {'Sq Feet':^12} {'Floors':^10} {'Overtime':^10}")
    print("-" * 80)
    
    for i in range(num_attendants):
        # Calculate statistics for each attendant
        hours = sum(df.loc[j,'Cleaning_Time_Hours'] * x[i,j].x for j in range(len(df)))
        rooms = sum(x[i,j].x for j in range(len(df)))
        sq_feet = sum(df.loc[j,'Square_Feet'] * x[i,j].x for j in range(len(df)))
        floors = sum(f[i,k].x for k in range(num_floors))
        overtime = sum(o[i,j].x for j in range(max_overtime))
        
        print(f"{i+1:^10d} {hours:^10.2f} {int(rooms):^10d} {sq_feet:^12.1f} {int(floors):^10.0f} {overtime:^10.1f}")
    
    print("-" * 80)
    
else:
    print("No optimal solution found")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 316 rows, 552 columns and 2568 nonzeros
Model fingerprint: 0x848c427d
Variable types: 0 continuous, 552 integer (552 binary)
Coefficient statistics:
  Matrix range     [6e-01, 2e+04]
  Objective range  [4e+01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+03]
Found heuristic solution: objective 3900.0000000
Presolve time: 0.00s
Presolved: 316 rows, 552 columns, 2568 nonzeros
Variable types: 0 continuous, 552 integer (552 binary)
Found heuristic solution: objective 3825.0000000

Root relaxation: objective 1.660511e+03, 159 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1660.51136    0   48 3825.00000 166

PART F

In [8]:
# Store original results
if model.status == GRB.OPTIMAL:
    binary_cost = model.objVal
    
    # Get binary model metrics
    binary_overtime = sum(o[i,j].x for i in range(num_attendants) for j in range(max_overtime))
    binary_floor_violations = sum(max(0, sum(f[i,k].x for k in range(num_floors)) - 2) 
                                for i in range(num_attendants))
    binary_sq_ft_violations = sum(s[i].x for i in range(num_attendants))

In [ ]:
relaxed_model = model.relax()
relaxed_model.optimize()

# Print comparison
if model.status == GRB.OPTIMAL and relaxed_model.status == GRB.OPTIMAL:
    print("\nComparison between Binary and Relaxed Models:")
    print("-" * 60)
    print(f"{'Metric':<30} {'Binary':<15} {'Relaxed':<15}")
    print("-" * 60)
    print(f"{'Optimal Cost':<30} ${binary_cost:<14.2f} ${relaxed_model.objVal:<14.2f}")
    print(f"{'Cost Difference':<30} {'':<15} ${binary_cost - relaxed_model.objVal:<14.2f}")
    print(f"{'Cost Gap (%)':<30} {'':<15} {((binary_cost - relaxed_model.objVal)/binary_cost)*100:<14.2f}%")
    print("-" * 60)

    
else:
    print("Optimal solution not found for one or both models")